# Census and Yelp API's 
### Data Sources:
[yelp developers](https://www.yelp.com/developers/documentation/v3)

[United States Census Bureau](https://www.census.gov/developers/)

--------------------------------------------

# Zip Code Data from the Census API (2017)

In [17]:
from census import Census
from us import states

from config import census_key
import numpy as np
import pandas as pd
import time

import json


In [3]:
c13 = Census(census_key, year=2013)

In [4]:
c17 = Census(census_key, year=2017)

# E: extraction

In [5]:
#data on all zip codes for 2017
_17census_data = c17.acs5.get(("B01003_001E", "B23025_005E"), {
                         'for': 'zip code tabulation area:*'})

In [ ]:
# Convert to DataFrame
_17_pd = pd.DataFrame(_17census_data)

In [8]:
_17_pd.head()

,B01003_001E,B23025_005E,zip code tabulation area
0,17599.0,2454.0,00601
1,39209.0,2538.0,00602
2,50135.0,3588.0,00603
3,6304.0,204.0,00606
4,27590.0,1474.0,00610


# T: transform

In [9]:
# Column Reordering
_17_pd = _17_pd.rename(columns={"B01003_001E": "Population",
                                      "B23025_005E": "Unemployment Count",
                                      "zip code tabulation area": "Zipcode"})

In [11]:
_17_pd.head()

,Population,Unemployment Count,Zipcode
0,17599.0,2454.0,00601
1,39209.0,2538.0,00602
2,50135.0,3588.0,00603
3,6304.0,204.0,00606
4,27590.0,1474.0,00610


In [12]:
# Add Employment Rate

_17_pd["Unemployment Rate"] = 100 * \
    _17_pd["Unemployment Count"].astype(
        int) / _17_pd["Population"].astype(int)

In [13]:
_17_pd.head()

,Population,Unemployment Count,Zipcode,Unemployment Rate
0,17599.0,2454.0,00601,13.943974
1,39209.0,2538.0,00602,6.473004
2,50135.0,3588.0,00603,7.156677
3,6304.0,204.0,00606,3.236041
4,27590.0,1474.0,00610,5.342515


In [14]:
_17_pd = _17_pd[["Zipcode", "Population","Unemployment Count", "Unemployment Rate"]]

In [15]:
_17_pd.head()

,Zipcode,Population,Unemployment Count,Unemployment Rate
0,00601,17599.0,2454.0,13.943974
1,00602,39209.0,2538.0,6.473004
2,00603,50135.0,3588.0,7.156677
3,00606,6304.0,204.0,3.236041
4,00610,27590.0,1474.0,5.342515


In [ ]:
_17_pd.head()

# L:  load
-----------------------------------------------
* Save df to csv format

In [18]:
_17_pd.to_csv("Resources/censusZips2017.csv", index=False)


### Create mongodb DATABASE 
* This DB will hold 2 tables: census and yelp

In [66]:
import pymongo

In [67]:
# Setup connection to mongodb
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [70]:
# Create  database 
db = client.census_yelp

### Create census collection

In [70]:
census_collection = db.census_zips_2017

In [71]:
censusDict = _17_pd.to_dict(orient='records')

In [72]:
census_collection.insert_many(censusDict)

In [91]:
# test query mongo db
census_query = census_collection.find()
census_query

In [92]:
for result in census_query[:2]:
    print()
    print(result)


{'_id': ObjectId('5ce06ddbbe1a7737c6489894'), 'Zipcode': '00601', 'Population': 17599.0, 'Unemployment Count': 2454.0, 'Unemployment Rate': 13.9439740894369}

{'_id': ObjectId('5ce06ddbbe1a7737c6489895'), 'Zipcode': '00602', 'Population': 39209.0, 'Unemployment Count': 2538.0, 'Unemployment Rate': 6.473003647121835}


### Create Yelp Collection

In [82]:
#Import Yelp data from csv file into pandas
yelp_data = pd.read_csv(
    "Resources/la_yelp_1000.csv", dtype="object", encoding="utf-8")

In [83]:
# Create Yelp collection to add to census_yelp mongo db
yelp_collection = db.yelp_biz_profiles

In [84]:
# transform/prepare  yelp_data data into a dictionary to load onto mongo
yelpDict = yelp_data.to_dict(orient='records')

In [85]:
yelp_collection.insert_many(yelpDict)

In [89]:
# test query mongo db
yelp_query = yelp_collection.find()
yelp_query

In [90]:
# Test query the yelp collenction
for result in yelp_query[:2]:
    print()
    print(result)


{'_id': ObjectId('5ce06ffbbe1a7737c64919f4'), 'id': 'TkFEKhsCixPWlShULKvMdQ', 'alias': 'bottega-louie-los-angeles', 'name': 'Bottega Louie', 'categories': 'Italian, Bakeries, Breakfast & Brunch', 'review_count': '16243', 'rating': '4.0', 'lat': '34.0469300995766', 'long': '-118.256601457672', 'address': '700 S Grand Ave Los Angeles, CA 90017', 'city': 'Los Angeles', 'zip_code': '90017', 'country': 'US', 'state': 'CA', 'phone': '+12138021470'}

{'_id': ObjectId('5ce06ffbbe1a7737c64919f5'), 'id': '7O1ORGY36A-2aIENyaJWPg', 'alias': 'howlin-rays-los-angeles-3', 'name': "Howlin' Ray's", 'categories': 'Southern, Chicken Shop, American (Traditional)', 'review_count': '5044', 'rating': '4.5', 'lat': '34.0614861063899', 'long': '-118.239554800093', 'address': '727 N Broadway Ste 128 Los Angeles, CA 90012', 'city': 'Los Angeles', 'zip_code': '90012', 'country': 'US', 'state': 'CA', 'phone': '+12139358399'}


### Example Using  SQL database from  Dataframe: 1 table

In [108]:
from sqlalchemy import create_engine

In [109]:
database_path = "Resources/us_census.sqlite"

In [110]:
engine = create_engine(f"sqlite:///{database_path}")

In [115]:
engine.table_names()

[]

In [116]:
_17_pd.to_sql(name='us_census_zips', con=engine, if_exists='append', index=False)

In [117]:
engine.table_names()

['us_census_zips']

In [119]:
pd.read_sql_query('select * from us_census_zips', con=engine).head()

,Zipcode,Population,Unemployment Count,Unemployment Rate
0,00601,17599.0,2454.0,13.943974
1,00602,39209.0,2538.0,6.473004
2,00603,50135.0,3588.0,7.156677
3,00606,6304.0,204.0,3.236041
4,00610,27590.0,1474.0,5.342515


### Test Query the biz_profile table

In [118]:
pd.read_sql_query('select * from us_census_zips', con=engine).head()

,Zipcode,Population,Unemployment Count,Unemployment Rate
0,00601,17599.0,2454.0,13.943974
1,00602,39209.0,2538.0,6.473004
2,00603,50135.0,3588.0,7.156677
3,00606,6304.0,204.0,3.236041
4,00610,27590.0,1474.0,5.342515


## Combine Yelp and Census Data

In [41]:
#Import Yelp data from csv file into pandas
yelp_data = pd.read_csv(
    "Resources/la_yelp_1000.csv", dtype="object", encoding="utf-8")

In [63]:
yelp_data.head()

,id,alias,name,categories,review_count,rating,lat,long,address,city,Zipcode,country,state,phone
0,TkFEKhsCixPWlShULKvMdQ,bottega-louie-los-angeles,Bottega Louie,"Italian, Bakeries, Breakfast & Brunch",16243,4.0,34.0469300995766,-118.256601457672,"700 S Grand Ave Los Angeles, CA 90017",Los Angeles,90017,US,CA,+12138021470
1,7O1ORGY36A-2aIENyaJWPg,howlin-rays-los-angeles-3,Howlin' Ray's,"Southern, Chicken Shop, American (Traditional)",5037,4.5,34.0614861063899,-118.239554800093,"727 N Broadway Ste 128 Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12139358399
2,KQBGm5G8IDkE8LeNY45mbA,wurstküche-los-angeles-2,Wurstküche,"Hot Dogs, German, Gastropubs",8058,4.0,34.0456047058105,-118.236061096191,"800 E 3rd St Los Angeles, CA 90013",Los Angeles,90013,US,CA,+12136874444
3,iSZpZgVnASwEmlq0DORY2A,daikokuya-little-tokyo-los-angeles,Daikokuya Little Tokyo,"Ramen, Noodles",8122,4.0,34.05008090944,-118.2401804513,"327 E 1st St Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12136261680
4,DJek3FUewBzMc0gS-Gms9w,the-morrison-los-angeles,The Morrison,"Gastropubs, Burgers, Bars",4072,4.5,34.12384,-118.26868,"3179 Los Feliz Blvd Los Angeles, CA 90039",Los Angeles,90039,US,CA,+13236671839


In [46]:
yelp_data.rename(columns={"zip_code": "Zipcode"}, inplace=True)

In [47]:
yelp_data.head()

,id,alias,name,categories,review_count,rating,lat,long,address,city,Zipcode,country,state,phone
0,TkFEKhsCixPWlShULKvMdQ,bottega-louie-los-angeles,Bottega Louie,"Italian, Bakeries, Breakfast & Brunch",16243,4.0,34.0469300995766,-118.256601457672,"700 S Grand Ave Los Angeles, CA 90017",Los Angeles,90017,US,CA,+12138021470
1,7O1ORGY36A-2aIENyaJWPg,howlin-rays-los-angeles-3,Howlin' Ray's,"Southern, Chicken Shop, American (Traditional)",5037,4.5,34.0614861063899,-118.239554800093,"727 N Broadway Ste 128 Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12139358399
2,KQBGm5G8IDkE8LeNY45mbA,wurstküche-los-angeles-2,Wurstküche,"Hot Dogs, German, Gastropubs",8058,4.0,34.0456047058105,-118.236061096191,"800 E 3rd St Los Angeles, CA 90013",Los Angeles,90013,US,CA,+12136874444
3,iSZpZgVnASwEmlq0DORY2A,daikokuya-little-tokyo-los-angeles,Daikokuya Little Tokyo,"Ramen, Noodles",8122,4.0,34.05008090944,-118.2401804513,"327 E 1st St Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12136261680
4,DJek3FUewBzMc0gS-Gms9w,the-morrison-los-angeles,The Morrison,"Gastropubs, Burgers, Bars",4072,4.5,34.12384,-118.26868,"3179 Los Feliz Blvd Los Angeles, CA 90039",Los Angeles,90039,US,CA,+13236671839


## Merge the two data sets on zip code

In [48]:
yelp_census_combined = pd.merge(
    yelp_data, _17_pd, how="left", on=["Zipcode", "Zipcode"])

In [64]:
yelp_census_combined.head()

,id,alias,name,categories,review_count,rating,lat,long,address,city,Zipcode,country,state,phone,Population,Unemployment Count,Unemployment Rate
0,TkFEKhsCixPWlShULKvMdQ,bottega-louie-los-angeles,Bottega Louie,"Italian, Bakeries, Breakfast & Brunch",16243,4.0,34.0469300995766,-118.256601457672,"700 S Grand Ave Los Angeles, CA 90017",Los Angeles,90017,US,CA,+12138021470,26390.0,920.0,3.486169
1,7O1ORGY36A-2aIENyaJWPg,howlin-rays-los-angeles-3,Howlin' Ray's,"Southern, Chicken Shop, American (Traditional)",5037,4.5,34.0614861063899,-118.239554800093,"727 N Broadway Ste 128 Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12139358399,33783.0,973.0,2.880147
2,KQBGm5G8IDkE8LeNY45mbA,wurstküche-los-angeles-2,Wurstküche,"Hot Dogs, German, Gastropubs",8058,4.0,34.0456047058105,-118.236061096191,"800 E 3rd St Los Angeles, CA 90013",Los Angeles,90013,US,CA,+12136874444,11957.0,817.0,6.832818
3,iSZpZgVnASwEmlq0DORY2A,daikokuya-little-tokyo-los-angeles,Daikokuya Little Tokyo,"Ramen, Noodles",8122,4.0,34.05008090944,-118.2401804513,"327 E 1st St Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12136261680,33783.0,973.0,2.880147
4,DJek3FUewBzMc0gS-Gms9w,the-morrison-los-angeles,The Morrison,"Gastropubs, Burgers, Bars",4072,4.5,34.12384,-118.26868,"3179 Los Feliz Blvd Los Angeles, CA 90039",Los Angeles,90039,US,CA,+13236671839,29092.0,1299.0,4.465145


# Save the Combined Data Frame as a csv

In [50]:
yelp_census_combined.to_csv(
    "Resources/yelp_census_data.csv", encoding="utf-8", index=False)

In [51]:
yelp_census_combined.head()

,id,alias,name,categories,review_count,rating,lat,long,address,city,Zipcode,country,state,phone,Population,Unemployment Count,Unemployment Rate
0,TkFEKhsCixPWlShULKvMdQ,bottega-louie-los-angeles,Bottega Louie,"Italian, Bakeries, Breakfast & Brunch",16243,4.0,34.0469300995766,-118.256601457672,"700 S Grand Ave Los Angeles, CA 90017",Los Angeles,90017,US,CA,+12138021470,26390.0,920.0,3.486169
1,7O1ORGY36A-2aIENyaJWPg,howlin-rays-los-angeles-3,Howlin' Ray's,"Southern, Chicken Shop, American (Traditional)",5037,4.5,34.0614861063899,-118.239554800093,"727 N Broadway Ste 128 Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12139358399,33783.0,973.0,2.880147
2,KQBGm5G8IDkE8LeNY45mbA,wurstküche-los-angeles-2,Wurstküche,"Hot Dogs, German, Gastropubs",8058,4.0,34.0456047058105,-118.236061096191,"800 E 3rd St Los Angeles, CA 90013",Los Angeles,90013,US,CA,+12136874444,11957.0,817.0,6.832818
3,iSZpZgVnASwEmlq0DORY2A,daikokuya-little-tokyo-los-angeles,Daikokuya Little Tokyo,"Ramen, Noodles",8122,4.0,34.05008090944,-118.2401804513,"327 E 1st St Los Angeles, CA 90012",Los Angeles,90012,US,CA,+12136261680,33783.0,973.0,2.880147
4,DJek3FUewBzMc0gS-Gms9w,the-morrison-los-angeles,The Morrison,"Gastropubs, Burgers, Bars",4072,4.5,34.12384,-118.26868,"3179 Los Feliz Blvd Los Angeles, CA 90039",Los Angeles,90039,US,CA,+13236671839,29092.0,1299.0,4.465145
